In [1]:
import geopandas as gpd
import numpy as np
from shapely.geometry import Point

# --- AYARLAR ---
# İki nokta arasındaki mesafe (metre cinsinden). Düşük değer daha hassas ama daha yavaş analiz demek.
GRID_SIZE_METERS = 150 
# Girdi ve Çıktı Dosyaları
INPUT_BOUNDARY_FILE = "yenimahalle_sinir.geojson"
OUTPUT_GRID_FILE = "yenimahalle_grid.geojson"


# 1. Ana sınırı oku
try:
    yenimahalle = gpd.read_file(INPUT_BOUNDARY_FILE)
except Exception as e:
    print(f"Hata: '{INPUT_BOUNDARY_FILE}' dosyası okunamadı. Lütfen dosyanın doğru yerde olduğundan emin ol. Hata: {e}")
    # Hata durumunda hücrenin çalışmasını durdur
    raise e

# Alan hesabı için uygun bir CRS'e geçelim (UTM)
yenimahalle_utm = yenimahalle.to_crs(epsg=32636)

# 2. Sınırların en uç noktalarını (bounding box) bul
minx, miny, maxx, maxy = yenimahalle_utm.total_bounds

# 3. Bu sınırlar içinde düzenli aralıklarla bir nokta ızgarası oluştur
x_coords = np.arange(minx, maxx, GRID_SIZE_METERS)
y_coords = np.arange(miny, maxy, GRID_SIZE_METERS)
grid_points = [Point(x, y) for x in x_coords for y in y_coords]

grid_gdf = gpd.GeoDataFrame(geometry=grid_points, crs="EPSG:32636")

# 4. Sadece Yenimahalle sınırları İÇİNDE kalan noktaları tut
# sjoin ile coğrafi birleştirme yaparak bunu kolayca yapabiliriz
points_within = gpd.sjoin(grid_gdf, yenimahalle_utm, how="inner", predicate='within')

# İhtiyacımız olmayan fazla sütunları atalım
points_within = points_within[['geometry']]

# 5. Sonucu tekrar standart enlem/boylam formatına (WGS84) çevir
final_grid = points_within.to_crs(epsg=4326)

# 6. Oluşturulan ızgarayı yeni bir GeoJSON dosyasına kaydet
final_grid.to_file(OUTPUT_GRID_FILE, driver='GeoJSON')

print(f"İşlem tamamlandı!")
print(f"Yenimahalle sınırları içine {GRID_SIZE_METERS} metre aralıklarla toplam {len(final_grid)} adet potansiyel nokta yerleştirildi.")
print(f"Sonuçlar '{OUTPUT_GRID_FILE}' dosyasına kaydedildi.")

İşlem tamamlandı!
Yenimahalle sınırları içine 150 metre aralıklarla toplam 10318 adet potansiyel nokta yerleştirildi.
Sonuçlar 'yenimahalle_grid.geojson' dosyasına kaydedildi.
